In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Global Variables
watchlist = ['AMZN', 'C', 'AAPL', 'IBM', 'CSCO', 'NVDA', 'MSFT', 'TM', 'CVX',
             'GOOG', 'FB', 'TSLA', 'JPM', 'JNJ', 'BAC', 'RLLCF', 'WMT', 'MA',
             'V', 'DIS', 'PG', 'RSPU', 'SQQQ', 'SLV', 'IWM', 'XSD', 'USO',
             'ECH', 'SPYV', 'GLD']

start_date = "2023-11-05"
end_date = "2024-11-05"
portfolio_size = 100000
sequence_length = 60  # For LSTM


C:\Users\ELENA\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Data Preprocessing

In [4]:
# Fetch historical prices
def fetch_data(watchlist, start_date, end_date):
    data = {}
    for ticker in watchlist:
        try:
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            data[ticker] = stock_data['Close']
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(data)

# Prepare daily returns
def calculate_returns(prices):
    return prices.pct_change().dropna()

prices = fetch_data(watchlist, start_date, end_date)
returns = calculate_returns(prices)


C:\Users\ELENA\anaconda3\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\Users\ELENA\anaconda3\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\Users\ELENA\anaconda3\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  

### ARIMA automation

In [7]:
# ARIMA Forecasting for a stock
def arima_forecast(series):
    try:
        model = ARIMA(series, order=(1, 1, 0))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=1)[0]
        return forecast
    except Exception as e:
        print(f"ARIMA failed for series: {e}")
        return np.nan

# Apply ARIMA to all securities
def arima_forecasts(returns):
    forecasts = {}
    for ticker in returns.columns:
        forecasts[ticker] = arima_forecast(returns[ticker])
    return pd.Series(forecasts)

arima_results = arima_forecasts(returns)


C:\Users\ELENA\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ELENA\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ELENA\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ELENA\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\ELENA\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning:

### LSTM automation

In [12]:
# Create sequences for LSTM
def create_sequences(data, sequence_length=60):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

# Build LSTM model
def build_lstm(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50, return_sequences=False),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# LSTM Forecast for a stock
def lstm_forecast(series):
    try:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

        # Train-Test Split
        train_size = int(len(scaled_data) * 0.8)
        train_data = scaled_data[:train_size]
        test_data = scaled_data[train_size-sequence_length:]

        X_train, y_train = create_sequences(train_data, sequence_length)
        X_test, y_test = create_sequences(test_data, sequence_length)

        # Model Training
        model = build_lstm((X_train.shape[1], 1))
        model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=0)

        # Forecast
        prediction = model.predict(X_test[-1].reshape(1, -1, 1))
        return scaler.inverse_transform(prediction)[0][0]
    except Exception as e:
        print(f"LSTM failed for series: {e}")
        return np.nan

# Apply LSTM to all securities
def lstm_forecasts(prices):
    forecasts = {}
    for ticker in prices.columns:
        forecasts[ticker] = lstm_forecast(prices[ticker])
    return pd.Series(forecasts)

lstm_results = lstm_forecasts(prices)


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 815ms/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step


### Combine Forecasts

In [15]:
# Weighted combination of ARIMA and LSTM results
def combine_forecasts(arima_results, lstm_results, alpha=0.5):
    return alpha * arima_results + (1 - alpha) * lstm_results

combined_forecasts = combine_forecasts(arima_results, lstm_results)


### Portfolio Construction

In [18]:
# Rank securities and allocate weights
def construct_portfolio(forecasts, prices, portfolio_size):
    ranked_forecasts = forecasts.sort_values(ascending=False)[:10]
    weights = ranked_forecasts / ranked_forecasts.sum()

    closing_prices = prices.iloc[-1][ranked_forecasts.index]
    shares = (weights * portfolio_size / closing_prices).apply(lambda x: int(x))

    portfolio = pd.DataFrame({
        "Ticker": ranked_forecasts.index,
        "Forecast": ranked_forecasts.values,
        "Weight": weights.values,
        "Closing Price": closing_prices.values,
        "Shares": shares.values,
        "Investment": shares * closing_prices
    })
    return portfolio

initial_portfolio = construct_portfolio(combined_forecasts, prices, portfolio_size)
print(initial_portfolio)


     Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  246.911301  0.176791     505.579987      34  17189.719543
MSFT   MSFT  212.591385  0.152218     408.459991      37  15113.019684
V         V  140.483887  0.100588     291.850006      34   9922.900208
GLD     GLD  123.053680  0.088108     252.830002      34   8596.220062
XSD     XSD  121.069191  0.086687     233.240005      37   8629.880203
TSLA   TSLA  118.795227  0.085059     242.839996      35   8499.399872
AAPL   AAPL  118.162460  0.084605     222.009995      38   8436.379791
IWM     IWM  106.786400  0.076460     220.089996      34   7483.059875
JPM     JPM  106.729774  0.076420     219.779999      34   7472.519958
IBM     IBM  102.045532  0.073066     206.320007      35   7221.200256


### Automate Rebalancing

In [21]:
# Rebalance portfolio
def rebalance_portfolio(portfolio, forecasts, prices, portfolio_value):
    new_portfolio = construct_portfolio(forecasts, prices, portfolio_value)
    changes = portfolio.merge(new_portfolio, on="Ticker", suffixes=("_old", "_new"))
    return new_portfolio, changes

# Example rebalance
portfolio_value = initial_portfolio["Investment"].sum()
rebalance_forecasts = combined_forecasts  # Replace with updated forecasts
new_portfolio, changes = rebalance_portfolio(initial_portfolio, rebalance_forecasts, prices, portfolio_value)
print(new_portfolio)


     Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  246.911301  0.176791     505.579987      34  17189.719543
MSFT   MSFT  212.591385  0.152218     408.459991      36  14704.559692
V         V  140.483887  0.100588     291.850006      33   9631.050201
GLD     GLD  123.053680  0.088108     252.830002      34   8596.220062
XSD     XSD  121.069191  0.086687     233.240005      36   8396.640198
TSLA   TSLA  118.795227  0.085059     242.839996      34   8256.559875
AAPL   AAPL  118.162460  0.084605     222.009995      37   8214.369797
IWM     IWM  106.786400  0.076460     220.089996      34   7483.059875
JPM     JPM  106.729774  0.076420     219.779999      34   7472.519958
IBM     IBM  102.045532  0.073066     206.320007      34   7014.880249


### Updating

In [24]:
# Rebalancing dates
rebalance_dates = ["2024-11-19", "2024-12-03", "2024-12-17", "2024-12-24"]

# Portfolio to track changes
portfolio_history = []

# Function to fetch updated prices
def fetch_prices_to_date(watchlist, start_date, end_date):
    data = {}
    for ticker in watchlist:
        try:
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            data[ticker] = stock_data['Close']
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(data)


### Automate Rebalancing

In [27]:
# Initialize portfolio
current_portfolio = initial_portfolio.copy()

# Rebalancing loop
for date in rebalance_dates:
    print(f"\nRebalancing on {date}...")
    
    # Fetch updated prices up to the rebalancing date
    updated_prices = fetch_prices_to_date(watchlist, start_date, date)
    updated_returns = calculate_returns(updated_prices)
    
    # Recalculate forecasts
    arima_results = arima_forecasts(updated_returns)
    lstm_results = lstm_forecasts(updated_prices)
    combined_forecasts = combine_forecasts(arima_results, lstm_results)
    
    # Calculate current portfolio value
    current_portfolio_value = (current_portfolio["Shares"] * updated_prices.iloc[-1][current_portfolio["Ticker"]]).sum()
    
    # Rebalance the portfolio
    new_portfolio, changes = rebalance_portfolio(current_portfolio, combined_forecasts, updated_prices, current_portfolio_value)
    
    # Log the portfolio history
    portfolio_history.append({
        "Date": date,
        "Portfolio": new_portfolio,
        "Changes": changes
    })
    
    # Update the current portfolio
    current_portfolio = new_portfolio.copy()

# Final portfolio on 24.12.2024
final_portfolio = current_portfolio
final_value = final_portfolio["Investment"].sum()
print("\nFinal Portfolio on 24.12.2024:")
print(final_portfolio)
print(f"Final Portfolio Value: ${final_value:,.2f}")



Rebalancing on 2024-11-19...


C:\Users\ELENA\anaconda3\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\Users\ELENA\anaconda3\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\Users\ELENA\anaconda3\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step


C:\Users\ELENA\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step

Final Portfolio on 24.12.2024:
     Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  254.407730  0.166410     528.960022      33  17455.680725
MSFT   MSFT  217.141998  0.142034     435.250000      34  14798.500000
TSLA   TSLA  200.901382  0.131411     430.600006      32  13779.200195
V         V  149.709000  0.097926     317.220001      32  10151.040039
AAPL   AAPL  124.598938  0.081501     255.270004      33   8423.910141
XSD     XSD  122.629684  0.080213     254.679993      33   8404.439758
GLD     GLD  122.439705  0.080089     240.960007      34   8192.640228
JPM     JPM  113.149231  0.074012     238.389999      32   7628.479980
IBM     IBM  112.690575  0.073712     221.929993      34   7545.619751
IWM     IWM  111.133507  0.072693     221.410004      34   7527.940125
Final Portfolio Value: $103,907.45


In [29]:
# Modify rebalance logic
def rebalance_portfolio(portfolio, forecasts, prices, portfolio_value, min_change=0.3):
    # Step 1: Construct new portfolio based on updated forecasts
    new_portfolio = construct_portfolio(forecasts, prices, portfolio_value)
    
    # Step 2: Calculate changes in volume
    changes = portfolio.merge(new_portfolio, on="Ticker", suffixes=("_old", "_new"))
    changes["Volume Change"] = abs(changes["Investment_old"] - changes["Investment_new"])
    total_volume_change = changes["Volume Change"].sum()
    
    # Step 3: Ensure at least 30% of the portfolio value is changed
    if total_volume_change < portfolio_value * min_change:
        print("Rebalancing does not meet the 30% volume change requirement. Adjusting...")
        # Example: Force the inclusion of top 2 new stocks
        top_new_stocks = forecasts.sort_values(ascending=False).index[:2]
        forced_stocks = construct_portfolio(forecasts[top_new_stocks], prices, portfolio_value)
        return forced_stocks, changes
    
    return new_portfolio, changes


In [31]:
print("\nPortfolio after rebalancing:")
print(new_portfolio)
print("\nChanges during rebalancing:")
print(changes)


Portfolio after rebalancing:
     Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  254.407730  0.166410     528.960022      33  17455.680725
MSFT   MSFT  217.141998  0.142034     435.250000      34  14798.500000
TSLA   TSLA  200.901382  0.131411     430.600006      32  13779.200195
V         V  149.709000  0.097926     317.220001      32  10151.040039
AAPL   AAPL  124.598938  0.081501     255.270004      33   8423.910141
XSD     XSD  122.629684  0.080213     254.679993      33   8404.439758
GLD     GLD  122.439705  0.080089     240.960007      34   8192.640228
JPM     JPM  113.149231  0.074012     238.389999      32   7628.479980
IBM     IBM  112.690575  0.073712     221.929993      34   7545.619751
IWM     IWM  111.133507  0.072693     221.410004      34   7527.940125

Changes during rebalancing:
  Ticker  Forecast_old  Weight_old  Closing Price_old  Shares_old  \
0     MA    252.650452    0.165702         530.309998          34   
1   MSFT    218.134567

### Rebalanced portfolios

In [44]:
print("\nPortfolio on 19.11.2024: ")
print(portfolio_history[0])


Portfolio on 19.11.2024: 
{'Date': '2024-11-19', 'Portfolio':      Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  251.880371  0.175776     521.630005      35  18257.050171
MSFT   MSFT  211.669128  0.147714     415.760010      37  15383.120361
V         V  145.187820  0.101320     312.160004      33  10301.280121
TSLA   TSLA  144.118515  0.100574     338.739990      30  10162.199707
GLD     GLD  121.118729  0.084523     241.089996      36   8679.239868
XSD     XSD  118.369408  0.082604     228.589996      37   8457.829865
AAPL   AAPL  115.047653  0.080286     228.020004      36   8208.720154
JPM     JPM  111.274574  0.077653     245.029999      33   8085.989960
IWM     IWM  110.827820  0.077342     228.929993      35   8012.549744
IBM     IBM  103.472183  0.072208     208.089996      36   7491.239868, 'Changes':   Ticker  Forecast_old  Weight_old  Closing Price_old  Shares_old  \
0     MA    246.911301    0.176791         505.579987          34   
1   MS

In [46]:
print("\nPortfolio on 03.12.2023: ")
print(portfolio_history[1])


Portfolio on 03.12.2023: 
{'Date': '2024-12-03', 'Portfolio':      Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  250.773849  0.172160     531.359985      34  18066.239502
MSFT   MSFT  210.419876  0.144457     430.980011      35  15084.300385
TSLA   TSLA  157.129135  0.107872     357.089996      32  11426.879883
V         V  148.159973  0.101714     316.649994      34  10766.099792
GLD     GLD  118.927216  0.081645     243.440002      35   8520.400085
XSD     XSD  117.890663  0.080934     252.679993      34   8591.119751
JPM     JPM  116.163223  0.079748     246.250000      34   8372.500000
AAPL   AAPL  115.345932  0.079187     239.589996      35   8385.649872
IWM     IWM  114.425369  0.078555     241.750000      34   8219.500000
IBM     IBM  107.394394  0.073728     227.389999      34   7731.259979, 'Changes':   Ticker  Forecast_old  Weight_old  Closing Price_old  Shares_old  \
0     MA    251.880371    0.175776         521.630005          35   
1   MS

In [48]:
print("\nPortfolio on 17.12.2024: ")
print(portfolio_history[2])


Portfolio on 17.12.2024: 
{'Date': '2024-12-17', 'Portfolio':      Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  252.650452  0.165702     530.309998      34  18030.539917
MSFT   MSFT  218.134567  0.143064     451.589996      34  15354.059875
TSLA   TSLA  196.056564  0.128584     463.019989      30  13890.599670
V         V  149.806747  0.098251     315.890015      34  10740.260498
XSD     XSD  125.302513  0.082180     265.160004      33   8750.280121
AAPL   AAPL  121.834396  0.079905     251.039993      34   8535.359772
GLD     GLD  120.807533  0.079232     244.880005      35   8570.800171
JPM     JPM  115.981552  0.076067     239.580002      34   8145.720062
IWM     IWM  113.674667  0.074554     234.440002      34   7970.960083
IBM     IBM  110.482292  0.072460     229.330002      34   7797.220062, 'Changes':   Ticker  Forecast_old  Weight_old  Closing Price_old  Shares_old  \
0     MA    250.773849    0.172160         531.359985          34   
1   MS

In [33]:
print("\nFinal Portfolio on 24.12.2024:")
print(final_portfolio)
print(f"Final Portfolio Value: ${final_value:,.2f}")


Final Portfolio on 24.12.2024:
     Ticker    Forecast    Weight  Closing Price  Shares    Investment
MA       MA  254.407730  0.166410     528.960022      33  17455.680725
MSFT   MSFT  217.141998  0.142034     435.250000      34  14798.500000
TSLA   TSLA  200.901382  0.131411     430.600006      32  13779.200195
V         V  149.709000  0.097926     317.220001      32  10151.040039
AAPL   AAPL  124.598938  0.081501     255.270004      33   8423.910141
XSD     XSD  122.629684  0.080213     254.679993      33   8404.439758
GLD     GLD  122.439705  0.080089     240.960007      34   8192.640228
JPM     JPM  113.149231  0.074012     238.389999      32   7628.479980
IBM     IBM  112.690575  0.073712     221.929993      34   7545.619751
IWM     IWM  111.133507  0.072693     221.410004      34   7527.940125
Final Portfolio Value: $103,907.45


### Our Profit

We gained $3907.45 

### Some Visualizations

In [53]:
!pip install jupyter-dash dash pandas plotly

  Obtaining dependency information for jupyter-dash from https://files.pythonhosted.org/packages/38/62/2c51aeb18c6d765c03c911e4d70f029ccb9de1a991d88cf30e362d2433c9/jupyter_dash-0.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/72/ef/d46131f4817f18b329e4fb7c53ba1d31774239d91266a74bccdc932708cc/dash-2.18.2-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from https://files.pythonhosted.org/packages/8f/04/9e36f28be4c0532c0e9207ff9dc01fb13a2b0eb036476a213b0000837d0e/retrying-1.3.4-py3-none-any.whl.metadata
  Obtaining dependency information for ansi2html from https://files.pythonhosted.org/packages/bd/71/aee71b836e9ee2741d5694b80d74bfc7c8cd5dbdf7a9f3035fcf80d792b1/ansi2html-1.9.2-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_comp

In [120]:
!pip install --upgrade jupyter-dash dash

In [67]:
# Example portfolio details from script
portfolio_history1 = []

for record in portfolio_history:  # Assuming `portfolio_history_data` is the output of your script
    portfolio_df = record["Portfolio"]
    changes_df = record["Changes"]
    date = record["Date"]
    
    # Append to portfolio history
    portfolio_history1.append({
        "Date": date,
        "Portfolio": portfolio_df,
        "Changes": changes_df
    })


In [134]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from dash import Dash, html

# Example Portfolio Data (Replace with actual data from your script)
portfolio_history1 = [
    {
        "Date": "2024-11-19",
        "Portfolio": pd.DataFrame({
            "Ticker": ["MA", "MSFT", "TSLA", "V", "XSD", "AAPL", "GLD", "JPM", "IWM", "IBM"],
            "Weight": [0.175776, 0.147714, 0.101320, 0.100574, 0.084523, 0.082604, 0.080286, 0.077653, 0.077342, 0.072208],
            "Investment": [18257.05, 15383.12, 10301.28, 10162.19, 8679.23, 8457.82, 8208.72, 8085.98, 8012.54, 7491.23],
            "Shares": [35, 37, 33, 30, 36, 37, 36, 33, 35, 36]
        }),
        # "Changes": pd.DataFrame({
        #     "Ticker": ["AAPL", "MSFT", "NVDA"],
        #     "Volume Change": [5000, -2000, 3000]
        # }),
    },
    {
        "Date": "2024-12-03",
        "Portfolio": pd.DataFrame({
            "Ticker": ["MA", "MSFT", "TSLA", "V", "XSD", "AAPL", "GLD", "JPM", "IWM", "IBM"],
            "Weight": [0.172160, 0.144457, 0.107872, 0.101714, 0.081645, 0.080934, 0.079748, 0.079187, 0.078555, 0.073728],
            "Investment": [18066.23, 15084.30, 11426.87, 10766.09, 8520.40, 8591.11, 8372.50, 8385.64, 8219.5,  7731.26],
            "Shares": [34, 35, 32, 34, 35, 34, 34, 35, 34, 34]
        }),
        # "Changes": pd.DataFrame({
        #     "Ticker": ["AAPL", "MSFT", "GOOG"],
        #     "Volume Change": [10000, -5000, 2000]
        # }),
    },
    {
        "Date": "2024-12-17",
        "Portfolio": pd.DataFrame({
            "Ticker": ["MA", "MSFT", "TSLA", "V", "XSD", "AAPL", "GLD", "JPM", "IWM", "IBM"],
            "Weight": [0.165702, 0.143064, 0.128584, 0.098251, 0.082180, 0.079905, 0.079232, 0.076067, 0.074554, 0.072460],
            "Investment": [18030.54, 15354.06, 13890.6, 10740.26, 8750.28, 8535.36, 8570.80, 8145.72, 7970.96, 7797.22],
            "Shares": [34, 34, 30, 34, 33, 34, 35, 34, 34, 34]
        }),
    },
    {
      "Date": "2024-12-24",
        "Portfolio": pd.DataFrame({
            "Ticker": ["MA", "MSFT", "TSLA", "V", "XSD", "AAPL", "GLD", "JPM", "IWM", "IBM"],
            "Weight": [0.166410, 0.142034, 0.131411, 0.097926, 0.081501, 0.080213, 0.080089, 0.074012, 0.073712, 0.072693],
            "Investment": [17455.68, 14798.5, 13779.2, 10151.04, 8423.91, 8404.4, 8192.64, 7628.5, 7545.62, 7527.94],
            "Shares": [33, 34, 32, 32, 33, 33, 34, 32, 34, 34]
        }),  
    }
]

# Extract portfolio values for a time series plot
portfolio_values = [
    {"Date": "2024-11-19", "Value": 103040},
    {"Date": "2024-12-03", "Value": 105160},
    {"Date": "2024-12-17", "Value": 107790},
    {"Date": "2024-12-24", "Value": 103907.45}
]

# Create Dash App
app = Dash(__name__)

app.css.config.serve_locally = True
app.scripts.config.serve_locally = True

# Layout
app.layout = html.Div([
    html.H1("Portfolio Rebalancing Dashboard", style={"textAlign": "center"}),
    
    # Dropdown to select rebalancing date
    html.Label("Select Rebalancing Date:"),
    dcc.Dropdown(
        id="date-dropdown",
        options=[{"label": item["Date"], "value": item["Date"]} for item in portfolio_history1],
        value=portfolio_history1[0]["Date"]
    ),
    
    # Portfolio details
    html.Div(id="portfolio-details"),
    
    # Graphs
    dcc.Graph(id="portfolio-pie"),
    dcc.Graph(id="portfolio-value-line"),
])

# Callbacks
@app.callback(
    [Output("portfolio-details", "children"),
     Output("portfolio-pie", "figure"),
     Output("portfolio-value-line", "figure")],
    [Input("date-dropdown", "value")]
)
def update_dashboard(selected_date):
    # Get portfolio for selected date
    portfolio_data = next(item for item in portfolio_history1 if item["Date"] == selected_date)
    portfolio = portfolio_data["Portfolio"]
    
    # Portfolio details table
    portfolio_table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in portfolio.columns])),
        html.Tbody([html.Tr([html.Td(portfolio.iloc[i][col]) for col in portfolio.columns])
                    for i in range(len(portfolio))])
    ])
    
    # Pie chart for portfolio allocation
    pie_fig = px.pie(portfolio, names="Ticker", values="Weight",
                     title=f"Portfolio Allocation on {selected_date}")
    
    # Line chart for portfolio value
    value_df = pd.DataFrame(portfolio_values)
    line_fig = px.line(value_df, x="Date", y="Value", title="Portfolio Value Over Time")
    
    return portfolio_table, pie_fig, line_fig


In [136]:
# Run the app
app.run_server(mode="external")

In [116]:
import ctypes

def kill(self):
    thread_id = self.get_id()
    print(f"Thread ID: {thread_id}")  # Debugging line
    if thread_id is None:
        raise ValueError("Thread ID is None!")
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        ctypes.c_long(thread_id), ctypes.py_object(SystemExit)
    )
    if res == 0:
        raise ValueError(f"Invalid thread id: {thread_id}")
